## Import main functions

In [0]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from config import imw, imh, c, RS
from config import repo_path

In [0]:
import seaborn as sns
import matplotlib.patheffects as PathEffects

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
from dataset_helpers import combine_images, efficient_shuffle

## Read images

In [0]:
npzfile = np.load('/Volumes/TARDIS/dmso_unmodified.npz')

In [0]:
x = npzfile['x']

In [0]:
filenames = npzfile['y']

In [0]:
x_combined, y_combined = combine_images(x, filenames, mask=False)

## Encode images

In [0]:
encoder = load_model(repo_path + '/data/processed/weights/encoder.h5', compile=False)

In [0]:
efficient_shuffle(x_combined, y_combined, random_state=RS)

In [ ]:
encoded_imgs = encoder.predict(x_combined)

## High dimensional visualisation to GIF
has to be done with t-sne because of the patch

**source code adapted from: https://github.com/oreillymedia/t-SNE-tutorial**

In [0]:
# monkey patch gradient descent 
import sklearn

positions = []

def _gradient_descent(objective, p0, it, n_iter,
                      n_iter_check=1, n_iter_without_progress=300,
                      momentum=0.8, learning_rate=200.0, min_gain=0.01,
                      min_grad_norm=1e-7, verbose=0, args=None, kwargs=None):

    if args is None:
        args = []
    if kwargs is None:
        kwargs = {}

    p = p0.copy().ravel()
    update = np.zeros_like(p)
    gains = np.ones_like(p)
    error = np.finfo(np.float).max
    best_error = np.finfo(np.float).max
    best_iter = i = it

    for i in range(it, n_iter):
        positions.append(p.copy())
        
        check_convergence = (i + 1) % n_iter_check == 0
        # only compute the error when needed
        kwargs['compute_error'] = check_convergence or i == n_iter - 1

        error, grad = objective(p, *args, **kwargs)
        grad_norm = linalg.norm(grad)

        inc = update * grad < 0.0
        dec = np.invert(inc)
        gains[inc] += 0.2
        gains[dec] *= 0.8
        np.clip(gains, min_gain, np.inf, out=gains)
        grad *= gains
        update = momentum * update - learning_rate * grad
        p += update

        if check_convergence:

            if verbose >= 2:
                print("[t-SNE] Iteration %d: error = %.7f,"
                      " gradient norm = %.7f"
                      " (%s iterations in %0.3fs)"
                      % (i + 1, error, grad_norm, n_iter_check, duration))

            if error < best_error:
                best_error = error
                best_iter = i
            elif i - best_iter > n_iter_without_progress:
                if verbose >= 2:
                    print("[t-SNE] Iteration %d: did not make any progress "
                          "during the last %d episodes. Finished."
                          % (i + 1, n_iter_without_progress))
                break
            if grad_norm <= min_grad_norm:
                if verbose >= 2:
                    print("[t-SNE] Iteration %d: gradient norm %f. Finished."
                          % (i + 1, grad_norm))
                break

    return p, error, i

sklearn.manifold.t_sne._gradient_descent = _gradient_descent

In [0]:
def scatter_gif(x, colors, labels=["Unstimulated", "OVA", "ConA"]):
    palette = np.array(sns.color_palette("hls", len(labels)))
    colors = np.array(colors)
    
    f = plt.figure(figsize=(8,8))
    ax = plt.subplot()
    
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[colors.astype(np.int)])
    
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    return f, sc

In [0]:
from scipy import linalg

from moviepy.video.io.bindings import mplfig_to_npimage
import moviepy.editor as mpy

In [0]:
def make_gif(out, x, y, labels=["Unstimulated", "OVA", "ConA", "Faulty"]):
    def make_frame_mpl(t):
        i = int(t*40)
        x = X_iter[..., i]
        sc.set_offsets(x)
        return mplfig_to_npimage(f)

    X_proj = TSNE(random_state=RS).fit_transform(x)
    X_iter = np.dstack(position.reshape(-1, 2) for position in positions)
    
    f, sc = scatter_gif(X_iter[..., -1], y, labels=labels)
    
    animation = mpy.VideoClip(make_frame_mpl,
                          duration=X_iter.shape[2]/40.)
    animation.write_gif(out, fps=20)

In [0]:
make_gif('/content/gdrive/My Drive/l4_dataset/cluster_combined_1.gif', encoded_imgs, y_combined)